# Cybersecurity Attack Type Detection - EDA
## Focus: Proxy + IP Trends, Spoofing Detection, and Data Bin Trends

**Team Member:** [Your Name]  
**Date:** January 31, 2026  
**Dataset:** 40,000 rows, 25 features

---
## 1. Setup and Data Loading

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (15, 8)
plt.rcParams['font.size'] = 10

print("✓ Libraries imported successfully!")

In [ ]:
# Load dataset
# TODO: Update the filepath to your actual CSV file location
df = pd.read_csv('your_dataset.csv')

print(f"Dataset Shape: {df.shape}")
print(f"Total Records: {df.shape[0]:,}")
print(f"Total Features: {df.shape[1]}")

In [ ]:
# Display first few rows
df.head()

In [ ]:
# Data types and basic info
df.info()

In [ ]:
# Missing values analysis
missing_df = pd.DataFrame({
    'Missing_Count': df.isnull().sum(),
    'Percentage': (df.isnull().sum() / len(df)) * 100,
    'Distinct_Count': df.nunique(),
    'Distinct_Percentage': (df.nunique() / len(df)) * 100
}).sort_values('Missing_Count', ascending=False)

print("\n" + "="*80)
print("MISSING VALUES AND DISTINCTNESS ANALYSIS")
print("="*80)
print(missing_df)

In [ ]:
# Attack Type distribution
if 'Attack Type' in df.columns:
    print("\nAttack Type Distribution:")
    print(df['Attack Type'].value_counts())
    
    plt.figure(figsize=(12, 6))
    df['Attack Type'].value_counts().plot(kind='bar', color='steelblue', edgecolor='black')
    plt.title('Attack Type Distribution', fontsize=14, fontweight='bold')
    plt.xlabel('Attack Type')
    plt.ylabel('Count')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

---
## 2. PART 1: Proxy Information Analysis

**Key Insights from Data Profiling:**
- 50% missing values (19,851 out of 40,000)
- 20,148 distinct values when present (highly diverse)
- This suggests proxy info is present only for certain attacks/sources

In [ ]:
# Proxy Information Analysis
print("="*80)
print("PROXY INFORMATION ANALYSIS")
print("="*80)

if 'Proxy Information' in df.columns:
    # Basic statistics
    total_records = len(df)
    proxy_present = df['Proxy Information'].notna().sum()
    proxy_missing = df['Proxy Information'].isna().sum()
    unique_proxies = df['Proxy Information'].nunique()
    
    print(f"\nProxy Information Statistics:")
    print(f"  - Total records: {total_records:,}")
    print(f"  - Records WITH proxy info: {proxy_present:,} ({proxy_present/total_records*100:.2f}%)")
    print(f"  - Records WITHOUT proxy info: {proxy_missing:,} ({proxy_missing/total_records*100:.2f}%)")
    print(f"  - Unique proxy values: {unique_proxies:,}")
    
    # Create binary feature: has_proxy
    df['has_proxy'] = df['Proxy Information'].notna().astype(int)
    
    print(f"\nProxy Usage Distribution:")
    print(df['has_proxy'].value_counts())
else:
    print("Warning: 'Proxy Information' column not found!")

In [ ]:
# Visualize proxy usage patterns
if 'has_proxy' in df.columns:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Overall proxy usage pie chart
    proxy_counts = df['has_proxy'].value_counts()
    labels = ['No Proxy', 'With Proxy']
    colors = ['lightcoral', 'lightgreen']
    axes[0, 0].pie(proxy_counts.values, labels=labels, autopct='%1.1f%%', 
                    colors=colors, startangle=90)
    axes[0, 0].set_title('Overall Proxy Usage Distribution', fontsize=14, fontweight='bold')
    
    # 2. Proxy usage by Attack Type
    if 'Attack Type' in df.columns:
        proxy_attack = pd.crosstab(df['Attack Type'], df['has_proxy'], normalize='index') * 100
        proxy_attack.plot(kind='bar', ax=axes[0, 1], stacked=False, 
                         color=['lightcoral', 'lightgreen'])
        axes[0, 1].set_title('Proxy Usage by Attack Type (%)', fontsize=14, fontweight='bold')
        axes[0, 1].set_xlabel('Attack Type')
        axes[0, 1].set_ylabel('Percentage')
        axes[0, 1].legend(['No Proxy', 'With Proxy'])
        axes[0, 1].tick_params(axis='x', rotation=45)
        
        # Print statistical summary
        print("\nProxy Usage by Attack Type:")
        print(proxy_attack)
    
    # 3. Proxy usage by Severity Level
    if 'Severity Level' in df.columns:
        proxy_severity = pd.crosstab(df['Severity Level'], df['has_proxy'])
        proxy_severity.plot(kind='bar', ax=axes[1, 0], color=['lightcoral', 'lightgreen'])
        axes[1, 0].set_title('Proxy Usage by Severity Level', fontsize=14, fontweight='bold')
        axes[1, 0].set_xlabel('Severity Level')
        axes[1, 0].set_ylabel('Count')
        axes[1, 0].legend(['No Proxy', 'With Proxy'])
        axes[1, 0].tick_params(axis='x', rotation=45)
    
    # 4. Proxy usage over time
    if 'Timestamp' in df.columns:
        df_temp = df.copy()
        df_temp['Timestamp'] = pd.to_datetime(df_temp['Timestamp'], errors='coerce')
        df_temp = df_temp.dropna(subset=['Timestamp'])
        df_temp['Date'] = df_temp['Timestamp'].dt.date
        
        proxy_time = df_temp.groupby('Date')['has_proxy'].agg(['sum', 'count'])
        proxy_time['percentage'] = (proxy_time['sum'] / proxy_time['count']) * 100
        
        axes[1, 1].plot(proxy_time.index, proxy_time['percentage'], 
                       marker='o', color='steelblue', linewidth=2)
        axes[1, 1].set_title('Proxy Usage Trend Over Time', fontsize=14, fontweight='bold')
        axes[1, 1].set_xlabel('Date')
        axes[1, 1].set_ylabel('Percentage Using Proxy')
        axes[1, 1].tick_params(axis='x', rotation=45)
        axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Analyze relationship with Log Source (Firewall vs Server)
if 'Log Source' in df.columns and 'has_proxy' in df.columns:
    print("\nProxy Usage by Log Source:")
    log_proxy = pd.crosstab(df['Log Source'], df['has_proxy'], normalize='index') * 100
    print(log_proxy)
    
    # Visualize
    log_proxy.plot(kind='bar', figsize=(10, 6), color=['lightcoral', 'lightgreen'])
    plt.title('Proxy Usage: Firewall vs Server Logs', fontsize=14, fontweight='bold')
    plt.xlabel('Log Source')
    plt.ylabel('Percentage')
    plt.legend(['No Proxy', 'With Proxy'])
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()

In [ ]:
# Statistical significance test
if 'Attack Type' in df.columns and 'has_proxy' in df.columns:
    print("\n" + "="*80)
    print("PROXY USAGE INSIGHTS BY ATTACK TYPE")
    print("="*80)
    
    for attack_type in df['Attack Type'].unique():
        subset = df[df['Attack Type'] == attack_type]
        proxy_pct = (subset['has_proxy'].sum() / len(subset)) * 100
        
        print(f"\n{attack_type}:")
        print(f"  - Total attacks: {len(subset):,}")
        print(f"  - With proxy: {subset['has_proxy'].sum():,} ({proxy_pct:.2f}%)")
        print(f"  - Without proxy: {len(subset) - subset['has_proxy'].sum():,} ({100-proxy_pct:.2f}%)")

### 📊 Key Findings - Proxy Analysis

**Summary:**
- Write your key findings here after running the cells above
- Which attack types use proxies most?
- Is there a correlation with severity?
- Any temporal patterns?

**Recommendation for ML Model:**
- The binary feature `has_proxy` appears to be a strong discriminator
- Consider as a key feature in your model

---
## 3. PART 2: IP Trends and Spoofing Detection

**Analysis Goals:**
1. Identify top source and destination IPs
2. Detect fan-out patterns (one source → many destinations = scanning/spoofing)
3. Detect fan-in patterns (many sources → one destination = DDoS)
4. Analyze bidirectional traffic
5. Detect private IP usage anomalies

In [ ]:
print("="*80)
print("IP TRENDS AND SPOOFING DETECTION")
print("="*80)

# Basic IP statistics
if 'Source IP Address' in df.columns and 'Destination IP Address' in df.columns:
    print(f"\nIP Address Statistics:")
    print(f"  - Unique Source IPs: {df['Source IP Address'].nunique():,}")
    print(f"  - Unique Destination IPs: {df['Destination IP Address'].nunique():,}")
    print(f"  - Total IP-to-IP connections: {len(df):,}")
    print(f"  - Average connections per source IP: {len(df)/df['Source IP Address'].nunique():.2f}")
    print(f"  - Average connections per destination IP: {len(df)/df['Destination IP Address'].nunique():.2f}")

In [ ]:
# Top Source IPs
print("\n" + "="*80)
print("TOP SOURCE IP ADDRESSES")
print("="*80)

top_src_ips = df['Source IP Address'].value_counts().head(20)
print("\nTop 20 Source IPs:")
print(top_src_ips)

# Visualize
plt.figure(figsize=(12, 8))
plt.barh(range(len(top_src_ips)), top_src_ips.values, color='steelblue')
plt.yticks(range(len(top_src_ips)), top_src_ips.index)
plt.xlabel('Frequency (Number of Connections)', fontsize=12)
plt.ylabel('Source IP Address', fontsize=12)
plt.title('Top 20 Most Active Source IP Addresses', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Top Destination IPs
print("\n" + "="*80)
print("TOP DESTINATION IP ADDRESSES")
print("="*80)

top_dst_ips = df['Destination IP Address'].value_counts().head(20)
print("\nTop 20 Destination IPs:")
print(top_dst_ips)

# Visualize
plt.figure(figsize=(12, 8))
plt.barh(range(len(top_dst_ips)), top_dst_ips.values, color='coral')
plt.yticks(range(len(top_dst_ips)), top_dst_ips.index)
plt.xlabel('Frequency (Number of Connections)', fontsize=12)
plt.ylabel('Destination IP Address', fontsize=12)
plt.title('Top 20 Most Targeted Destination IP Addresses', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# SPOOFING DETECTION 1: Fan-out Analysis (Source IP → Multiple Destinations)
print("\n" + "="*80)
print("SPOOFING INDICATOR 1: FAN-OUT PATTERN (Source → Multiple Destinations)")
print("="*80)

# Count unique destinations per source IP
src_to_dst_mapping = df.groupby('Source IP Address')['Destination IP Address'].nunique()
src_to_dst_mapping = src_to_dst_mapping.sort_values(ascending=False)

# Calculate thresholds
threshold_95 = src_to_dst_mapping.quantile(0.95)
threshold_99 = src_to_dst_mapping.quantile(0.99)

suspicious_sources_95 = src_to_dst_mapping[src_to_dst_mapping > threshold_95]
suspicious_sources_99 = src_to_dst_mapping[src_to_dst_mapping > threshold_99]

print(f"\nFan-out Statistics:")
print(f"  - Mean destinations per source: {src_to_dst_mapping.mean():.2f}")
print(f"  - Median destinations per source: {src_to_dst_mapping.median():.2f}")
print(f"  - 95th percentile threshold: {threshold_95:.0f} destinations")
print(f"  - 99th percentile threshold: {threshold_99:.0f} destinations")
print(f"\nSuspicious Source IPs:")
print(f"  - IPs above 95th percentile: {len(suspicious_sources_95)} ({len(suspicious_sources_95)/len(src_to_dst_mapping)*100:.2f}%)")
print(f"  - IPs above 99th percentile: {len(suspicious_sources_99)} ({len(suspicious_sources_99)/len(src_to_dst_mapping)*100:.2f}%)")

print(f"\nTop 10 Source IPs with Highest Fan-out:")
print(src_to_dst_mapping.head(10))

In [ ]:
# Visualize fan-out distribution
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Histogram
axes[0].hist(src_to_dst_mapping.values, bins=50, color='red', alpha=0.7, edgecolor='black')
axes[0].axvline(threshold_95, color='darkred', linestyle='--', linewidth=2, 
                label=f'95th percentile: {threshold_95:.0f}')
axes[0].axvline(threshold_99, color='maroon', linestyle='--', linewidth=2, 
                label=f'99th percentile: {threshold_99:.0f}')
axes[0].set_xlabel('Number of Unique Destinations per Source IP', fontsize=12)
axes[0].set_ylabel('Frequency (log scale)', fontsize=12)
axes[0].set_title('Source IP Fan-out Distribution\n(Potential Scanning/Spoofing)', 
                  fontsize=14, fontweight='bold')
axes[0].set_yscale('log')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Top suspicious IPs
top_suspicious = src_to_dst_mapping.head(15)
axes[1].barh(range(len(top_suspicious)), top_suspicious.values, color='darkred')
axes[1].set_yticks(range(len(top_suspicious)))
axes[1].set_yticklabels(top_suspicious.index)
axes[1].set_xlabel('Number of Unique Destinations', fontsize=12)
axes[1].set_ylabel('Source IP Address', fontsize=12)
axes[1].set_title('Top 15 Source IPs by Fan-out\n(Most Suspicious)', 
                  fontsize=14, fontweight='bold')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

In [ ]:
# SPOOFING DETECTION 2: Fan-in Analysis (Multiple Sources → Single Destination)
print("\n" + "="*80)
print("SPOOFING INDICATOR 2: FAN-IN PATTERN (Multiple Sources → Destination)")
print("="*80)

# Count unique sources per destination IP
dst_to_src_mapping = df.groupby('Destination IP Address')['Source IP Address'].nunique()
dst_to_src_mapping = dst_to_src_mapping.sort_values(ascending=False)

# Calculate thresholds
threshold_95_dst = dst_to_src_mapping.quantile(0.95)
threshold_99_dst = dst_to_src_mapping.quantile(0.99)

suspicious_targets_95 = dst_to_src_mapping[dst_to_src_mapping > threshold_95_dst]
suspicious_targets_99 = dst_to_src_mapping[dst_to_src_mapping > threshold_99_dst]

print(f"\nFan-in Statistics:")
print(f"  - Mean sources per destination: {dst_to_src_mapping.mean():.2f}")
print(f"  - Median sources per destination: {dst_to_src_mapping.median():.2f}")
print(f"  - 95th percentile threshold: {threshold_95_dst:.0f} sources")
print(f"  - 99th percentile threshold: {threshold_99_dst:.0f} sources")
print(f"\nSuspicious Target IPs (Potential DDoS Victims):")
print(f"  - IPs above 95th percentile: {len(suspicious_targets_95)} ({len(suspicious_targets_95)/len(dst_to_src_mapping)*100:.2f}%)")
print(f"  - IPs above 99th percentile: {len(suspicious_targets_99)} ({len(suspicious_targets_99)/len(dst_to_src_mapping)*100:.2f}%)")

print(f"\nTop 10 Target IPs with Highest Fan-in:")
print(dst_to_src_mapping.head(10))

In [ ]:
# Visualize fan-in distribution
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Histogram
axes[0].hist(dst_to_src_mapping.values, bins=50, color='purple', alpha=0.7, edgecolor='black')
axes[0].axvline(threshold_95_dst, color='darkviolet', linestyle='--', linewidth=2, 
                label=f'95th percentile: {threshold_95_dst:.0f}')
axes[0].axvline(threshold_99_dst, color='indigo', linestyle='--', linewidth=2, 
                label=f'99th percentile: {threshold_99_dst:.0f}')
axes[0].set_xlabel('Number of Unique Sources per Destination IP', fontsize=12)
axes[0].set_ylabel('Frequency (log scale)', fontsize=12)
axes[0].set_title('Destination IP Fan-in Distribution\n(Potential DDoS Targets)', 
                  fontsize=14, fontweight='bold')
axes[0].set_yscale('log')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Top targeted IPs
top_targets = dst_to_src_mapping.head(15)
axes[1].barh(range(len(top_targets)), top_targets.values, color='darkviolet')
axes[1].set_yticks(range(len(top_targets)))
axes[1].set_yticklabels(top_targets.index)
axes[1].set_xlabel('Number of Unique Sources', fontsize=12)
axes[1].set_ylabel('Destination IP Address', fontsize=12)
axes[1].set_title('Top 15 Destination IPs by Fan-in\n(Potential DDoS Targets)', 
                  fontsize=14, fontweight='bold')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

In [ ]:
# SPOOFING DETECTION 3: Bidirectional Traffic Analysis
print("\n" + "="*80)
print("SPOOFING INDICATOR 3: BIDIRECTIONAL TRAFFIC")
print("="*80)

source_ips_set = set(df['Source IP Address'].dropna())
dest_ips_set = set(df['Destination IP Address'].dropna())
bidirectional_ips = source_ips_set.intersection(dest_ips_set)

print(f"\nBidirectional IP Statistics:")
print(f"  - Total unique source IPs: {len(source_ips_set):,}")
print(f"  - Total unique destination IPs: {len(dest_ips_set):,}")
print(f"  - IPs appearing as BOTH source and destination: {len(bidirectional_ips):,}")
print(f"  - Percentage of bidirectional IPs: {len(bidirectional_ips)/(len(source_ips_set.union(dest_ips_set)))*100:.2f}%")

# Analyze bidirectional traffic by attack type
if 'Attack Type' in df.columns:
    df['is_bidirectional'] = (df['Source IP Address'].isin(bidirectional_ips)) | \
                              (df['Destination IP Address'].isin(bidirectional_ips))
    
    print(f"\nBidirectional Traffic by Attack Type:")
    bidir_attack = pd.crosstab(df['Attack Type'], df['is_bidirectional'], normalize='index') * 100
    print(bidir_attack)

In [ ]:
# Visualize bidirectional traffic
if 'Attack Type' in df.columns and 'is_bidirectional' in df.columns:
    plt.figure(figsize=(12, 6))
    bidir_attack[True].sort_values().plot(kind='barh', color='teal', edgecolor='black')
    plt.xlabel('Percentage of Traffic with Bidirectional IPs', fontsize=12)
    plt.ylabel('Attack Type', fontsize=12)
    plt.title('Bidirectional IP Traffic by Attack Type', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

In [ ]:
# SPOOFING DETECTION 4: Private IP Detection
print("\n" + "="*80)
print("SPOOFING INDICATOR 4: PRIVATE IP ADDRESS DETECTION")
print("="*80)

def is_private_ip(ip):
    """Check if an IP is in private range (RFC 1918)"""
    if pd.isna(ip):
        return False
    try:
        parts = str(ip).split('.')
        if len(parts) != 4:
            return False
        first = int(parts[0])
        second = int(parts[1])
        
        # Private IP ranges: 10.x.x.x, 172.16-31.x.x, 192.168.x.x
        if first == 10:
            return True
        if first == 172 and 16 <= second <= 31:
            return True
        if first == 192 and second == 168:
            return True
        return False
    except:
        return False

df['src_is_private'] = df['Source IP Address'].apply(is_private_ip)
df['dst_is_private'] = df['Destination IP Address'].apply(is_private_ip)

print(f"\nPrivate IP Statistics:")
print(f"  - Source IPs from private ranges: {df['src_is_private'].sum():,} ({df['src_is_private'].sum()/len(df)*100:.2f}%)")
print(f"  - Destination IPs from private ranges: {df['dst_is_private'].sum():,} ({df['dst_is_private'].sum()/len(df)*100:.2f}%)")
print(f"  - Total connections involving private IPs: {(df['src_is_private'] | df['dst_is_private']).sum():,}")

In [ ]:
# Analyze private IP usage by attack type
if 'Attack Type' in df.columns:
    print("\nPrivate IP Usage by Attack Type:")
    attack_private = df.groupby('Attack Type').agg({
        'src_is_private': ['sum', 'mean'],
        'dst_is_private': ['sum', 'mean']
    })
    attack_private.columns = ['Src_Private_Count', 'Src_Private_Pct', 'Dst_Private_Count', 'Dst_Private_Pct']
    attack_private['Src_Private_Pct'] = attack_private['Src_Private_Pct'] * 100
    attack_private['Dst_Private_Pct'] = attack_private['Dst_Private_Pct'] * 100
    print(attack_private)
    
    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    attack_private['Src_Private_Pct'].plot(kind='bar', ax=axes[0], color='orange', edgecolor='black')
    axes[0].set_title('Source Private IP Usage by Attack Type', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Attack Type')
    axes[0].set_ylabel('Percentage')
    axes[0].tick_params(axis='x', rotation=45)
    
    attack_private['Dst_Private_Pct'].plot(kind='bar', ax=axes[1], color='red', edgecolor='black')
    axes[1].set_title('Destination Private IP Usage by Attack Type', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Attack Type')
    axes[1].set_ylabel('Percentage')
    axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Geo-location analysis (if available)
if 'Geo-location Data' in df.columns:
    print("\n" + "="*80)
    print("GEO-LOCATION ANALYSIS")
    print("="*80)
    
    print(f"\nGeo-location Statistics:")
    print(f"  - Unique locations: {df['Geo-location Data'].nunique():,}")
    print(f"  - Missing values: {df['Geo-location Data'].isna().sum():,}")
    
    # Top locations
    print(f"\nTop 15 Geo-locations:")
    top_locations = df['Geo-location Data'].value_counts().head(15)
    print(top_locations)
    
    # Visualize
    plt.figure(figsize=(14, 8))
    top_locations.plot(kind='barh', color='skyblue', edgecolor='black')
    plt.xlabel('Frequency', fontsize=12)
    plt.ylabel('Geo-location', fontsize=12)
    plt.title('Top 15 Geo-locations in Attack Traffic', fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    # Geo-location by attack type
    if 'Attack Type' in df.columns:
        print(f"\nTop Geo-location by Attack Type:")
        for attack in df['Attack Type'].unique():
            top_loc = df[df['Attack Type'] == attack]['Geo-location Data'].value_counts().head(1)
            if len(top_loc) > 0:
                print(f"  {attack}: {top_loc.index[0]} ({top_loc.values[0]} occurrences)")

### 📊 Key Findings - IP Trends & Spoofing

**Summary:**
- Write your key findings here
- How many suspicious IPs detected (fan-out/fan-in)?
- Any DDoS targets identified?
- Private IP issues?
- Geographic patterns?

**Red Flags Identified:**
- List specific suspicious IPs or patterns

**Recommendation for ML Model:**
- Create features: source_fanout_score, dest_fanin_score, is_bidirectional, is_private_ip

---
## 4. PART 3: Data Bin Trends Analysis

**Analysis Goals:**
1. Packet Length distribution and binning
2. Port usage patterns (well-known, registered, dynamic)
3. Protocol distribution
4. Anomaly score categorization

In [ ]:
print("="*80)
print("DATA BIN TRENDS ANALYSIS")
print("="*80)

In [ ]:
# 1. PACKET LENGTH ANALYSIS
print("\n" + "="*80)
print("PACKET LENGTH DISTRIBUTION")
print("="*80)

if 'Packet Length' in df.columns:
    packet_lengths = df['Packet Length'].dropna()
    
    print(f"\nPacket Length Statistics:")
    print(f"  - Mean: {packet_lengths.mean():.2f} bytes")
    print(f"  - Median: {packet_lengths.median():.2f} bytes")
    print(f"  - Std Dev: {packet_lengths.std():.2f} bytes")
    print(f"  - Min: {packet_lengths.min():.2f} bytes")
    print(f"  - Max: {packet_lengths.max():.2f} bytes")
    print(f"  - 25th percentile: {packet_lengths.quantile(0.25):.2f} bytes")
    print(f"  - 75th percentile: {packet_lengths.quantile(0.75):.2f} bytes")

In [ ]:
# Create packet length bins
bins_packet = [0, 64, 128, 256, 512, 1024, 2048, float('inf')]
labels_packet = ['0-64', '64-128', '128-256', '256-512', '512-1024', '1024-2048', '2048+']
df['packet_length_bin'] = pd.cut(df['Packet Length'], bins=bins_packet, labels=labels_packet)

packet_bin_dist = df['packet_length_bin'].value_counts().sort_index()
print(f"\nPacket Length Bins Distribution:")
print(packet_bin_dist)
print(f"\nPercentage Distribution:")
print((packet_bin_dist / packet_bin_dist.sum() * 100).round(2))

In [ ]:
# Visualize packet length
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Histogram
axes[0, 0].hist(packet_lengths, bins=50, color='skyblue', edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Packet Length (bytes)', fontsize=11)
axes[0, 0].set_ylabel('Frequency (log scale)', fontsize=11)
axes[0, 0].set_title('Packet Length Distribution', fontsize=13, fontweight='bold')
axes[0, 0].set_yscale('log')
axes[0, 0].grid(True, alpha=0.3)

# Binned distribution
packet_bin_dist.plot(kind='bar', ax=axes[0, 1], color='coral', edgecolor='black')
axes[0, 1].set_xlabel('Packet Length Bins (bytes)', fontsize=11)
axes[0, 1].set_ylabel('Count', fontsize=11)
axes[0, 1].set_title('Packet Length Binned Distribution', fontsize=13, fontweight='bold')
axes[0, 1].tick_params(axis='x', rotation=45)

# Box plot by attack type
if 'Attack Type' in df.columns:
    df.boxplot(column='Packet Length', by='Attack Type', ax=axes[1, 0])
    axes[1, 0].set_xlabel('Attack Type', fontsize=11)
    axes[1, 0].set_ylabel('Packet Length (bytes)', fontsize=11)
    axes[1, 0].set_title('Packet Length by Attack Type', fontsize=13, fontweight='bold')
    axes[1, 0].get_figure().suptitle('')  # Remove default title
    plt.sca(axes[1, 0])
    plt.xticks(rotation=45, ha='right')

# Bins by attack type (stacked bar)
if 'Attack Type' in df.columns:
    bin_attack = pd.crosstab(df['Attack Type'], df['packet_length_bin'], normalize='index') * 100
    bin_attack.plot(kind='bar', stacked=True, ax=axes[1, 1], colormap='tab10')
    axes[1, 1].set_xlabel('Attack Type', fontsize=11)
    axes[1, 1].set_ylabel('Percentage', fontsize=11)
    axes[1, 1].set_title('Packet Length Bins by Attack Type (%)', fontsize=13, fontweight='bold')
    axes[1, 1].legend(title='Packet Size', bbox_to_anchor=(1.05, 1), loc='upper left')
    axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# 2. PORT ANALYSIS
print("\n" + "="*80)
print("PORT USAGE ANALYSIS")
print("="*80)

if 'Source Port' in df.columns and 'Destination Port' in df.columns:
    # Source ports
    print(f"\nTop 10 Source Ports:")
    top_src_ports = df['Source Port'].value_counts().head(10)
    print(top_src_ports)
    
    # Destination ports
    print(f"\nTop 10 Destination Ports:")
    top_dst_ports = df['Destination Port'].value_counts().head(10)
    print(top_dst_ports)

In [ ]:
# Create port categories
def categorize_port(port):
    """Categorize ports into well-known, registered, or dynamic"""
    if pd.isna(port):
        return 'Unknown'
    try:
        port = int(port)
        if 0 <= port <= 1023:
            return 'Well-known (0-1023)'
        elif 1024 <= port <= 49151:
            return 'Registered (1024-49151)'
        elif 49152 <= port <= 65535:
            return 'Dynamic (49152-65535)'
        else:
            return 'Unknown'
    except:
        return 'Unknown'

df['dst_port_category'] = df['Destination Port'].apply(categorize_port)
df['src_port_category'] = df['Source Port'].apply(categorize_port)

print(f"\nDestination Port Categories:")
print(df['dst_port_category'].value_counts())
print(f"\nPercentage:")
print((df['dst_port_category'].value_counts() / len(df) * 100).round(2))

In [ ]:
# Visualize port analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Top source ports
top_src_ports_15 = df['Source Port'].value_counts().head(15)
axes[0, 0].barh(range(len(top_src_ports_15)), top_src_ports_15.values, color='lightgreen')
axes[0, 0].set_yticks(range(len(top_src_ports_15)))
axes[0, 0].set_yticklabels(top_src_ports_15.index)
axes[0, 0].set_xlabel('Frequency', fontsize=11)
axes[0, 0].set_ylabel('Port Number', fontsize=11)
axes[0, 0].set_title('Top 15 Source Ports', fontsize=13, fontweight='bold')
axes[0, 0].invert_yaxis()

# Top destination ports
top_dst_ports_15 = df['Destination Port'].value_counts().head(15)
axes[0, 1].barh(range(len(top_dst_ports_15)), top_dst_ports_15.values, color='lightcoral')
axes[0, 1].set_yticks(range(len(top_dst_ports_15)))
axes[0, 1].set_yticklabels(top_dst_ports_15.index)
axes[0, 1].set_xlabel('Frequency', fontsize=11)
axes[0, 1].set_ylabel('Port Number', fontsize=11)
axes[0, 1].set_title('Top 15 Destination Ports', fontsize=13, fontweight='bold')
axes[0, 1].invert_yaxis()

# Port category pie chart
port_cat_dist = df['dst_port_category'].value_counts()
axes[1, 0].pie(port_cat_dist.values, labels=port_cat_dist.index, autopct='%1.1f%%', startangle=90)
axes[1, 0].set_title('Destination Port Categories', fontsize=13, fontweight='bold')

# Port categories by attack type
if 'Attack Type' in df.columns:
    port_attack = pd.crosstab(df['Attack Type'], df['dst_port_category'])
    port_attack.plot(kind='bar', stacked=True, ax=axes[1, 1], colormap='Set3')
    axes[1, 1].set_xlabel('Attack Type', fontsize=11)
    axes[1, 1].set_ylabel('Count', fontsize=11)
    axes[1, 1].set_title('Port Categories by Attack Type', fontsize=13, fontweight='bold')
    axes[1, 1].legend(title='Port Category', bbox_to_anchor=(1.05, 1), loc='upper left')
    axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# 3. PROTOCOL ANALYSIS
print("\n" + "="*80)
print("PROTOCOL DISTRIBUTION")
print("="*80)

if 'Protocol' in df.columns:
    protocol_dist = df['Protocol'].value_counts()
    print(f"\nProtocol Distribution:")
    print(protocol_dist)
    print(f"\nPercentage:")
    print((protocol_dist / protocol_dist.sum() * 100).round(2))

In [ ]:
# Visualize protocol analysis
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Protocol pie chart
axes[0].pie(protocol_dist.values, labels=protocol_dist.index, autopct='%1.1f%%', startangle=90)
axes[0].set_title('Protocol Distribution', fontsize=14, fontweight='bold')

# Protocol by attack type
if 'Attack Type' in df.columns:
    protocol_attack = pd.crosstab(df['Attack Type'], df['Protocol'], normalize='index') * 100
    protocol_attack.plot(kind='bar', stacked=True, ax=axes[1], colormap='viridis')
    axes[1].set_xlabel('Attack Type', fontsize=12)
    axes[1].set_ylabel('Percentage', fontsize=12)
    axes[1].set_title('Protocol Distribution by Attack Type (%)', fontsize=14, fontweight='bold')
    axes[1].legend(title='Protocol', bbox_to_anchor=(1.05, 1), loc='upper left')
    axes[1].tick_params(axis='x', rotation=45)
    
    print(f"\nProtocol Usage by Attack Type (%):")
    print(protocol_attack.round(2))

plt.tight_layout()
plt.show()

In [ ]:
# 4. ANOMALY SCORES ANALYSIS
print("\n" + "="*80)
print("ANOMALY SCORES DISTRIBUTION")
print("="*80)

if 'Anomaly Scores' in df.columns:
    anomaly_scores = df['Anomaly Scores'].dropna()
    
    print(f"\nAnomaly Score Statistics:")
    print(f"  - Mean: {anomaly_scores.mean():.4f}")
    print(f"  - Median: {anomaly_scores.median():.4f}")
    print(f"  - Std Dev: {anomaly_scores.std():.4f}")
    print(f"  - Min: {anomaly_scores.min():.4f}")
    print(f"  - Max: {anomaly_scores.max():.4f}")
    print(f"  - 25th percentile: {anomaly_scores.quantile(0.25):.4f}")
    print(f"  - 50th percentile: {anomaly_scores.quantile(0.50):.4f}")
    print(f"  - 75th percentile: {anomaly_scores.quantile(0.75):.4f}")

In [ ]:
# Create anomaly score categories based on quartiles
anomaly_bins = [anomaly_scores.min(), 
                anomaly_scores.quantile(0.25),
                anomaly_scores.quantile(0.5),
                anomaly_scores.quantile(0.75),
                anomaly_scores.max()]
anomaly_labels = ['Low (0-25%)', 'Medium (25-50%)', 'High (50-75%)', 'Critical (75-100%)']

df['anomaly_category'] = pd.cut(df['Anomaly Scores'], bins=anomaly_bins, 
                                 labels=anomaly_labels, include_lowest=True)

anomaly_cat_dist = df['anomaly_category'].value_counts().sort_index()
print(f"\nAnomaly Score Categories:")
print(anomaly_cat_dist)
print(f"\nPercentage:")
print((anomaly_cat_dist / anomaly_cat_dist.sum() * 100).round(2))

In [ ]:
# Visualize anomaly scores
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Histogram
axes[0, 0].hist(anomaly_scores, bins=50, color='purple', alpha=0.7, edgecolor='black')
axes[0, 0].set_xlabel('Anomaly Score', fontsize=11)
axes[0, 0].set_ylabel('Frequency', fontsize=11)
axes[0, 0].set_title('Anomaly Score Distribution', fontsize=13, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# Category bar chart
colors = ['green', 'yellow', 'orange', 'red']
axes[0, 1].bar(range(len(anomaly_cat_dist)), anomaly_cat_dist.values, 
               color=colors, edgecolor='black')
axes[0, 1].set_xticks(range(len(anomaly_cat_dist)))
axes[0, 1].set_xticklabels(anomaly_cat_dist.index, rotation=45, ha='right')
axes[0, 1].set_xlabel('Anomaly Category', fontsize=11)
axes[0, 1].set_ylabel('Count', fontsize=11)
axes[0, 1].set_title('Anomaly Score Categories', fontsize=13, fontweight='bold')

# Box plot by attack type
if 'Attack Type' in df.columns:
    df.boxplot(column='Anomaly Scores', by='Attack Type', ax=axes[1, 0])
    axes[1, 0].set_xlabel('Attack Type', fontsize=11)
    axes[1, 0].set_ylabel('Anomaly Score', fontsize=11)
    axes[1, 0].set_title('Anomaly Scores by Attack Type', fontsize=13, fontweight='bold')
    axes[1, 0].get_figure().suptitle('')
    plt.sca(axes[1, 0])
    plt.xticks(rotation=45, ha='right')

# Category by attack type
if 'Attack Type' in df.columns:
    anomaly_attack = pd.crosstab(df['Attack Type'], df['anomaly_category'], normalize='index') * 100
    anomaly_attack.plot(kind='bar', stacked=True, ax=axes[1, 1], color=colors)
    axes[1, 1].set_xlabel('Attack Type', fontsize=11)
    axes[1, 1].set_ylabel('Percentage', fontsize=11)
    axes[1, 1].set_title('Anomaly Categories by Attack Type (%)', fontsize=13, fontweight='bold')
    axes[1, 1].legend(title='Anomaly Level', bbox_to_anchor=(1.05, 1), loc='upper left')
    axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

### 📊 Key Findings - Data Bin Trends

**Summary:**
- Write your key findings here
- What are the dominant packet sizes per attack type?
- Which ports are most targeted?
- Protocol preferences?
- Anomaly score patterns?

**Attack Signatures Identified:**
- DDoS: [packet size pattern, protocol, ports]
- Malware: [packet size pattern, protocol, ports]
- etc.

**Recommendation for ML Model:**
- Use binned features: packet_length_bin, port_category, anomaly_category

---
## 5. COMPREHENSIVE SUMMARY & INSIGHTS

In [ ]:
print("\n" + "="*80)
print("COMPREHENSIVE EDA SUMMARY")
print("="*80)

print(f"\n📊 DATASET OVERVIEW")
print("-" * 80)
print(f"Total Records: {len(df):,}")
print(f"Total Features: {df.shape[1]}")
print(f"Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

if 'Attack Type' in df.columns:
    print(f"\n🎯 ATTACK TYPE DISTRIBUTION")
    print("-" * 80)
    attack_dist = df['Attack Type'].value_counts()
    for attack, count in attack_dist.items():
        print(f"  {attack}: {count:,} ({count/len(df)*100:.2f}%)")

print(f"\n🔍 KEY STATISTICS")
print("-" * 80)

# Proxy
if 'has_proxy' in df.columns:
    proxy_pct = (df['has_proxy'].sum() / len(df)) * 100
    print(f"  - Proxy Usage Rate: {proxy_pct:.2f}%")

# IPs
if 'Source IP Address' in df.columns:
    print(f"  - Unique Source IPs: {df['Source IP Address'].nunique():,}")
    print(f"  - Unique Destination IPs: {df['Destination IP Address'].nunique():,}")

# Packet Length
if 'Packet Length' in df.columns:
    print(f"  - Average Packet Size: {df['Packet Length'].mean():.2f} bytes")

# Protocol
if 'Protocol' in df.columns:
    top_protocol = df['Protocol'].value_counts().index[0]
    top_protocol_pct = (df['Protocol'].value_counts().values[0] / len(df)) * 100
    print(f"  - Most Common Protocol: {top_protocol} ({top_protocol_pct:.2f}%)")

# Port
if 'Destination Port' in df.columns:
    top_port = df['Destination Port'].value_counts().index[0]
    top_port_count = df['Destination Port'].value_counts().values[0]
    print(f"  - Most Targeted Port: {top_port} ({top_port_count:,} times)")

print("\n" + "="*80)
print("✅ EDA ANALYSIS COMPLETE!")
print("="*80)

---
## 6. EXPORT ENGINEERED FEATURES (Optional)

Create new features based on EDA insights for ML model

In [ ]:
# Create a summary of engineered features
engineered_features = df[[
    'has_proxy',                    # Binary: 0/1
    'is_bidirectional',            # Binary: 0/1  
    'src_is_private',              # Binary: 0/1
    'dst_is_private',              # Binary: 0/1
    'packet_length_bin',           # Categorical: 7 categories
    'dst_port_category',           # Categorical: 3 categories
    'src_port_category',           # Categorical: 3 categories
    'anomaly_category'             # Categorical: 4 categories
]].copy()

print("Engineered Features Summary:")
print(engineered_features.head(10))
print(f"\nShape: {engineered_features.shape}")
print(f"\nFeature Data Types:")
print(engineered_features.dtypes)

In [ ]:
# Optional: Save engineered features to CSV
# engineered_features.to_csv('engineered_features.csv', index=False)
# print("✓ Engineered features saved to 'engineered_features.csv'")

---
## 7. CONCLUSIONS & RECOMMENDATIONS

### Key Findings:
1. **Proxy Usage:**
   - [Your findings here]
   
2. **IP Spoofing Indicators:**
   - [Your findings here]
   
3. **Data Bin Patterns:**
   - [Your findings here]

### Recommendations for ML Model:
1. Binary features: `has_proxy`, `is_bidirectional`, `src_is_private`, `dst_is_private`
2. Categorical features: `packet_length_bin`, `port_category`, `anomaly_category`
3. Numerical features: Consider creating fan-out/fan-in scores
4. Attack-specific patterns identified can guide feature importance analysis

### Next Steps:
1. Data preprocessing (handle missing values, encode categoricals)
2. Feature scaling/normalization
3. Address class imbalance if needed
4. Model selection and training
5. Hyperparameter tuning
6. Model evaluation